In [1]:
import sys
sys.path.insert(0, '..')
%load_ext autoreload
%autoreload 2
%aimport std_func

## Mean-Variance Analysis - Minimum Variance

Mean-variance analysis is a mathematical framework that examplifies the trade-off between return and risk. It is used to create diversified portfolios based on investors’ expectation. There are one main approach used in this report. We have the minimum volatility portfolio that concentrates on minimizing the risk of the portfolio. Mimimum variance portfolios constructed can help us investigate if document embedding can reveal information on returns. 

Mimimum variance portfolio are constructed based on three estimates of covariance: 

a) the sample covariance of returns

b) the cosine similarity of the business descriptions

c) factor model based on business descriptions and return data

When we construct portfolios, to avoid high weights on certain companies, we set constraint that each company selected can only occupy the whole portfolio up to 20%. Also, we assume no short selling in the portfolios.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Monthly Returns
Since we will generate cosine similarity estimate and factor model estimate using business description of companies from 2016 to 2018, we only consider monthly returns before 2019. We group the monthly stock returns data extracted from Wharton Research Data Services’s (WRDS) CRSP/Compustat database by company name and date first. After a selection of time span, from June 2016 to December 2018, we are able to get 31-month returns for 719 companies.

In [3]:
r_selected = pd.read_csv("data/filtered_r.csv")
# get the mean of all 
r_selected.set_index("name", inplace = True)
mu = r_selected.mean(axis = 1)
# compute the covariance matrix 
cov = r_selected.T.cov()

In [4]:
r_selected

,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,...,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31
name,,,,,,,,,,,,,,,,,,,,,
1ST SOURCE CORP,-0.044261,0.043223,0.052960,0.008618,-0.026754,0.138889,0.134654,0.010076,0.039459,0.005138,...,0.026358,0.027262,0.027308,0.004701,0.058581,-0.005658,-0.060189,-0.114595,0.043357,-0.165840
"2U, INC.",0.056394,0.189391,0.010292,0.083475,-0.089580,-0.051635,-0.088022,0.129022,0.073737,0.085089,...,0.015100,-0.042128,0.177786,-0.118565,-0.094543,0.181073,-0.158572,-0.163320,-0.071849,-0.148484
3D SYSTEMS CORP,0.022405,-0.021914,0.082898,0.237931,-0.227298,-0.001442,-0.040433,0.240783,-0.078229,-0.015789,...,0.220000,-0.133736,0.231076,0.117314,-0.118755,0.672145,-0.071253,-0.360847,0.024834,-0.178514
ABBOTT LABORATORIES,-0.008075,0.145001,-0.061006,0.006426,-0.065973,-0.029817,0.008931,0.094376,0.079243,-0.014862,...,-0.006796,-0.025200,0.058490,-0.008776,0.079193,0.019835,0.097546,-0.056434,0.074122,-0.023228
ABBVIE INC.,-0.016208,0.078986,-0.032161,-0.016069,-0.106548,0.089996,0.029934,-0.013893,0.011946,0.053687,...,-0.182854,0.030217,0.024754,-0.063574,0.005828,0.040659,-0.014586,-0.166737,0.210918,-0.022064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIOPHARM ONCOLOGY INC,-0.295250,-0.114754,0.043210,0.110454,0.008881,0.140845,-0.174383,0.106542,0.072635,-0.001575,...,0.050938,0.076531,0.161137,-0.383673,-0.142384,0.127413,0.095890,-0.381250,0.691919,-0.441791
ZOETIS INC.,0.002847,0.063422,0.012483,0.017808,-0.080946,0.055962,0.062525,0.028302,-0.029669,0.001125,...,0.032773,0.001150,0.002635,0.017802,0.016622,0.047641,0.010596,-0.015400,0.042662,-0.088740
"ZOGENIX, INC.",-0.201389,0.134161,-0.044907,0.310780,-0.286964,0.546012,-0.035714,-0.337449,0.285714,0.048309,...,-0.055425,-0.018727,0.082697,0.038778,0.283937,-0.148899,0.026915,-0.158065,-0.016044,-0.112679


#### Sample Mean

In [5]:
mu

name
1ST SOURCE CORP                  0.009076
2U, INC.                         0.024622
3D SYSTEMS CORP                  0.009901
ABBOTT LABORATORIES              0.022606
ABBVIE INC.                      0.018973
                                   ...   
ZIOPHARM ONCOLOGY INC           -0.022408
ZOETIS INC.                      0.020733
ZOGENIX, INC.                    0.084996
ZOSANO PHARMA CORP              -0.045029
ZYNERBA PHARMACEUTICALS, INC.   -0.003030
Length: 719, dtype: float64

#### Sample Covariance

In [6]:
cov

name,1ST SOURCE CORP,"2U, INC.",3D SYSTEMS CORP,ABBOTT LABORATORIES,ABBVIE INC.,ABEONA THERAPEUTICS INC.,ABRAXAS PETROLEUM CORP,ACADIA PHARMACEUTICALS INC,ACADIA REALTY TRUST,ACELRX PHARMACEUTICALS INC,...,XBIOTECH INC.,XENCOR INC,XENON PHARMACEUTICALS INC.,XOMA CORP,"ZEDGE, INC.",ZIOPHARM ONCOLOGY INC,ZOETIS INC.,"ZOGENIX, INC.",ZOSANO PHARMA CORP,"ZYNERBA PHARMACEUTICALS, INC."
name,,,,,,,,,,,,,,,,,,,,,
1ST SOURCE CORP,0.004037,0.002210,0.002809,0.000770,0.002418,0.003306,0.005177,0.002316,0.001162,0.003289,...,-0.003115,0.003075,0.002632,0.004194,0.004987,0.005336,0.001121,0.009492,0.006947,0.007938
"2U, INC.",0.002210,0.011917,0.010599,0.001859,0.003255,0.008266,0.000459,0.002350,0.001803,0.007566,...,-0.002568,0.006610,0.002465,0.005714,0.002574,0.005000,0.001211,0.006308,0.006805,0.011199
3D SYSTEMS CORP,0.002809,0.010599,0.041611,0.002045,0.001621,0.003528,0.002866,0.000601,0.002762,0.008999,...,-0.004730,0.007320,0.016933,-0.010924,0.002617,0.010725,0.002968,0.003055,0.006912,0.016312
ABBOTT LABORATORIES,0.000770,0.001859,0.002045,0.002716,0.002223,0.000001,-0.001260,0.004953,0.000644,0.003408,...,-0.001995,0.000060,0.001042,0.005094,0.000346,0.002999,0.000776,0.003065,0.008432,0.003382
ABBVIE INC.,0.002418,0.003255,0.001621,0.002223,0.007276,0.003242,0.000717,0.003293,-0.000128,0.000062,...,-0.002213,0.004581,-0.002855,0.013811,0.008295,0.008498,0.001434,0.014807,0.003906,0.005539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIOPHARM ONCOLOGY INC,0.005336,0.005000,0.010725,0.002999,0.008498,0.016045,0.002711,0.009927,0.001973,0.008192,...,-0.002800,0.013566,-0.003391,0.011584,0.024891,0.045590,0.001967,0.004574,0.017793,0.012347
ZOETIS INC.,0.001121,0.001211,0.002968,0.000776,0.001434,-0.000931,0.001524,0.000304,0.000496,0.002061,...,-0.002579,0.001551,0.001696,0.001430,0.002570,0.001967,0.002037,0.001968,0.001314,0.004093
"ZOGENIX, INC.",0.009492,0.006308,0.003055,0.003065,0.014807,0.020618,0.013220,0.009577,0.001537,0.028441,...,-0.015704,0.007531,-0.003133,0.059678,0.009011,0.004574,0.001968,0.153881,0.024156,0.033034
